# Viewing databse data, filtering to Houston only, and Adding Coordinates
* Before running this code:
    * Run the removing trailing spaces sql code within the sql marketing schema
    * This update only needs to be ran once. If you done this already please disregard and continue with the below

In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import pandas.io.sql as pdsql
from sqlalchemy import create_engine, event, DateTime
from db_setup import mysql_user, mysql_password, db_name
import mysql.connector

#### Engin creation to MYSQL

In [2]:
connection_string = f"{mysql_user}:{mysql_password}@localhost:3306/{db_name}?charset=utf8"
engine = create_engine(f'mysql://{connection_string}')

#### Testing engine by viewing table

In [3]:
engine.table_names()

['building_account',
 'ccim',
 'franchise_tax_info',
 'franchise_tax_info_date',
 'ftoffdir',
 'sales_tax_info',
 'sales_tax_info_date']

## Fran from MYSQL

In [4]:
df_franchise_tax_info_date = pdsql.read_sql("SELECT * FROM franchise_tax_info_date",engine)
df_franchise_tax_info_date.head(3)

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Status_Date,Current_Exempt_Reason_Code,Agent_Name,Agent_Address,Agent_City,Agent_State,Agent_Zip_Code,Taxpayer_Organizational_Name,Record_Type_Name,DateTime
0,12020053802,"KAGER INDUSTRIES, LLC",3168 N STATE HIGHWAY 16,FREDERICKSBRG,TX,78624,86,CL,8309979391,U,...,2020-01-01 00:00:00,,ROBERT KAGER,3168 NORTH STATE HWY. 16,FREDERICKSBURG,TX,78624,Limited Liability Company - Texas,Secretary of State (SOS) File Number,2020-03-25 02:13:35
1,13415672842,FLAMING RIVER INDUSTRIES INC,800 POERTNER DR,BEREA,OH,44017,0,CF,4408264488,X,...,None,,,,,,,Foreign Profit,Comptroller Assigned File Number,2020-03-25 02:13:35
2,13513654320,LA SALLE COMPANY INC,4315 RALPH JONES CT,SOUTH BEND,IN,46628,0,CF,5742717644,X,...,None,,,,,,,Foreign Profit,Comptroller Assigned File Number,2020-03-25 02:13:35


#### Filtering Texas and Houston data only

In [5]:
df_franchise_tax_info_date= df_franchise_tax_info_date[(df_franchise_tax_info_date['Taxpayer_City']=='HOUSTON') & (df_franchise_tax_info_date['Taxpayer_State']=='TX')]
df_franchise_tax_info_date.count()

Taxpayer_Number                   12944
Taxpayer_Name                     12944
Taxpayer_Address                  12944
Taxpayer_City                     12944
Taxpayer_State                    12944
Taxpayer_Zip_Code                 12944
Taxpayer_County_Code              12944
Taxpayer_Organizational_Type      12944
Taxpayer_Phone_Number             12944
Record_Type_Code                  12944
Responsibility_Beginning_Date     12944
Secretary_of_State_File_Number    12944
SOS_Charter_Date                  12903
SOS_Status_Date                   12903
Current_Exempt_Reason_Code        12944
Agent_Name                        12944
Agent_Address                     12944
Agent_City                        12944
Agent_State                       12944
Agent_Zip_Code                    12944
Taxpayer_Organizational_Name      12578
Record_Type_Name                  12944
DateTime                          12944
dtype: int64

#### Selecting specific columns

In [6]:
df_franchise_tax_info_date = df_franchise_tax_info_date[['Taxpayer_Number','Taxpayer_Name','Taxpayer_Address','Taxpayer_City','Taxpayer_State','Taxpayer_Zip_Code','Taxpayer_Organizational_Type','Taxpayer_Organizational_Name','Record_Type_Code','Record_Type_Name','SOS_Charter_Date','SOS_Status_Date','Responsibility_Beginning_Date','DateTime']]
df_franchise_tax_info_date.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_Organizational_Type,Taxpayer_Organizational_Name,Record_Type_Code,Record_Type_Name,SOS_Charter_Date,SOS_Status_Date,Responsibility_Beginning_Date,DateTime
10,32050048308,"GULF COAST PRE-STRESS, INC.",3355 W ALABAMA ST STE 630,HOUSTON,TX,77098,CT,Texas Profit,U,Secretary of State (SOS) File Number,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-03-25 02:13:35
86,32072223376,SPRING BRANCH ISD EXPLORE POST 900,9009 RULAND RD,HOUSTON,TX,77055,CN,Texas Non-Profit,U,Secretary of State (SOS) File Number,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-03-25 02:13:35
115,32072296588,THE LOUIS L. BORICK FOUNDATION,2707 KIPLING ST,HOUSTON,TX,77098,CF,Foreign Profit,X,Comptroller Assigned File Number,None,None,2020-01-02 00:00:00,2020-03-25 02:13:35
124,32072316154,NDN GOLDCRAFT JEWELRIES LLC,3003 SEAGLER RD APT 6203,HOUSTON,TX,77042,CL,Limited Liability Company - Texas,U,Secretary of State (SOS) File Number,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-03-25 02:13:35
129,32072320818,POWER DESIGN AND ILLUMINATIONS LLC,138 TINA LN,HOUSTON,TX,77037,CL,Limited Liability Company - Texas,U,Secretary of State (SOS) File Number,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-03-25 02:13:35


#### Dropping duplicates

In [7]:
df_franchise_tax_info_date = df_franchise_tax_info_date.drop_duplicates(subset='Taxpayer_Number', keep='first')

#### Saving to csv file

In [8]:
formattedData = os.path.abspath('HotelOccupancyTaxData/formattedData/'+ 'HOUfran.csv')
df_franchise_tax_info_date.to_csv(formattedData, index = False)

## FTACT

In [9]:
df_franchise_tax_info = pdsql.read_sql("SELECT * FROM  franchise_tax_info WHERE Taxpayer_City = 'HOUSTON'AND Taxpayer_State = 'TX';",engine)
df_franchise_tax_info.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code,Taxpayer_Organizational_Name,Record_Type_Name,Responsibility_End_Reason_Name,SOS_Status_Name,Rigth_to_Tansact_Business_Name,DateTime
0,10104336481,FLOTATION TECHNOLOGIES INC,945 BUNKER HILL RD STE 500,HOUSTON,TX,77024,101,CF,2072827749,X,...,N,0,None,420000,Foreign Profit,Comptroller Assigned File Number,Active or Inactive with no Reason Code,None,Forfeited,2020-03-25 02:13:35
1,10105480411,"MOTORCYCLES UNLIMITED, INC.",1964 CYPRESS CREEK PKWY,HOUSTON,TX,77090,101,CT,2814440545,U,...,A,0,None,541213,Texas Profit,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Active,Active,2020-03-25 02:13:35
2,10105483688,"CYPRESS CREEK PERSONNEL SERVICES, INC.",13131 CHAMPIONS DR STE 202,HOUSTON,TX,77069,101,CT,2815802180,U,...,A,0,None,561000,Texas Profit,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Active,Active,2020-03-25 02:13:35
3,10105495120,"LIL IKES, INC.",7110 AVENUE C,HOUSTON,TX,77011,101,CT,7139210088,U,...,A,0,None,811121,Texas Profit,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Active,Active,2020-03-25 02:13:35
4,10105517915,"WOOD GROUP USA, INC.",17325 PARK ROW,HOUSTON,TX,77084,101,CT,2816478449,U,...,A,0,None,0,Texas Profit,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Active,Active,2020-03-25 02:13:35


#### Count of columns

In [10]:
df_franchise_tax_info.count()

Taxpayer_Number                   508118
Taxpayer_Name                     508116
Taxpayer_Address                  508118
Taxpayer_City                     508118
Taxpayer_State                    508118
Taxpayer_Zip_Code                 508118
Taxpayer_County_Code              508118
Taxpayer_Organizational_Type      508118
Taxpayer_Phone_Number             295306
Record_Type_Code                  508118
Responsibility_Beginning_Date     508118
Responsibility_End_Date                0
Responsibility_End_Reason_Code    508118
Secretary_of_State_File_Number    508118
SOS_Charter_Date                  493650
SOS_Status_Date                   493650
SOS_Status_Code                   493650
Rigth_to_Tansact_Business_Code    508118
Current_Exempt_Reason_Code        508118
Exempt_Begin_Date                  34770
NAICS_Code                        508118
Taxpayer_Organizational_Name      508118
Record_Type_Name                  508118
Responsibility_End_Reason_Name    508118
SOS_Status_Name 

#### Selecting specific columns

In [11]:
df_franchise_tax_info = df_franchise_tax_info[['Taxpayer_Number','Taxpayer_Name','Taxpayer_Address','Taxpayer_City','Taxpayer_State','Taxpayer_Zip_Code','Taxpayer_Organizational_Type','Taxpayer_Organizational_Name','Record_Type_Code','Record_Type_Name','Responsibility_Beginning_Date','Responsibility_End_Reason_Code','Responsibility_End_Reason_Name','SOS_Charter_Date','SOS_Status_Date','SOS_Status_Code','SOS_Status_Name','Rigth_to_Tansact_Business_Code','Rigth_to_Tansact_Business_Name','DateTime']]
df_franchise_tax_info.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_Organizational_Type,Taxpayer_Organizational_Name,Record_Type_Code,Record_Type_Name,Responsibility_Beginning_Date,Responsibility_End_Reason_Code,Responsibility_End_Reason_Name,SOS_Charter_Date,SOS_Status_Date,SOS_Status_Code,SOS_Status_Name,Rigth_to_Tansact_Business_Code,Rigth_to_Tansact_Business_Name,DateTime
0,10104336481,FLOTATION TECHNOLOGIES INC,945 BUNKER HILL RD STE 500,HOUSTON,TX,77024,CF,Foreign Profit,X,Comptroller Assigned File Number,2000-12-01 00:00:00,0,Active or Inactive with no Reason Code,None,None,None,None,N,Forfeited,2020-03-25 02:13:35
1,10105480411,"MOTORCYCLES UNLIMITED, INC.",1964 CYPRESS CREEK PKWY,HOUSTON,TX,77090,CT,Texas Profit,U,Secretary of State (SOS) File Number,2001-12-17 00:00:00,0,Active or Inactive with no Reason Code,2001-12-17 00:00:00,2001-12-17 00:00:00,A,Active,A,Active,2020-03-25 02:13:35
2,10105483688,"CYPRESS CREEK PERSONNEL SERVICES, INC.",13131 CHAMPIONS DR STE 202,HOUSTON,TX,77069,CT,Texas Profit,U,Secretary of State (SOS) File Number,2001-11-20 00:00:00,0,Active or Inactive with no Reason Code,2001-11-20 00:00:00,2001-11-20 00:00:00,A,Active,A,Active,2020-03-25 02:13:35
3,10105495120,"LIL IKES, INC.",7110 AVENUE C,HOUSTON,TX,77011,CT,Texas Profit,U,Secretary of State (SOS) File Number,2001-12-04 00:00:00,0,Active or Inactive with no Reason Code,2001-12-04 00:00:00,2001-12-04 00:00:00,A,Active,A,Active,2020-03-25 02:13:35
4,10105517915,"WOOD GROUP USA, INC.",17325 PARK ROW,HOUSTON,TX,77084,CT,Texas Profit,U,Secretary of State (SOS) File Number,2007-01-01 00:00:00,0,Active or Inactive with no Reason Code,2012-12-22 00:00:00,2012-12-22 00:00:00,A,Active,A,Active,2020-03-25 02:13:35


#### Dropping duplicate data

In [12]:
df_franchise_tax_info = df_franchise_tax_info.drop_duplicates(subset='Taxpayer_Number', keep='first')

#### Checking columns after dropping duplicates

In [13]:
df_franchise_tax_info.count()

Taxpayer_Number                   254054
Taxpayer_Name                     254053
Taxpayer_Address                  254054
Taxpayer_City                     254054
Taxpayer_State                    254054
Taxpayer_Zip_Code                 254054
Taxpayer_Organizational_Type      254054
Taxpayer_Organizational_Name      254054
Record_Type_Code                  254054
Record_Type_Name                  254054
Responsibility_Beginning_Date     254054
Responsibility_End_Reason_Code    254054
Responsibility_End_Reason_Name    254054
SOS_Charter_Date                  246820
SOS_Status_Date                   246820
SOS_Status_Code                   246820
SOS_Status_Name                   246812
Rigth_to_Tansact_Business_Code    254054
Rigth_to_Tansact_Business_Name    254054
DateTime                          254054
dtype: int64

#### Removing outlier from data by getting data only from 1980 and above

In [14]:
df_franchise_tax_info= df_franchise_tax_info[df_franchise_tax_info['Responsibility_Beginning_Date']>='1980-01-01']
df_franchise_tax_info.sort_values(by=['Responsibility_Beginning_Date'])
df_franchise_tax_info.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_Organizational_Type,Taxpayer_Organizational_Name,Record_Type_Code,Record_Type_Name,Responsibility_Beginning_Date,Responsibility_End_Reason_Code,Responsibility_End_Reason_Name,SOS_Charter_Date,SOS_Status_Date,SOS_Status_Code,SOS_Status_Name,Rigth_to_Tansact_Business_Code,Rigth_to_Tansact_Business_Name,DateTime
0,10104336481,FLOTATION TECHNOLOGIES INC,945 BUNKER HILL RD STE 500,HOUSTON,TX,77024,CF,Foreign Profit,X,Comptroller Assigned File Number,2000-12-01 00:00:00,0,Active or Inactive with no Reason Code,None,None,None,None,N,Forfeited,2020-03-25 02:13:35
1,10105480411,"MOTORCYCLES UNLIMITED, INC.",1964 CYPRESS CREEK PKWY,HOUSTON,TX,77090,CT,Texas Profit,U,Secretary of State (SOS) File Number,2001-12-17 00:00:00,0,Active or Inactive with no Reason Code,2001-12-17 00:00:00,2001-12-17 00:00:00,A,Active,A,Active,2020-03-25 02:13:35
2,10105483688,"CYPRESS CREEK PERSONNEL SERVICES, INC.",13131 CHAMPIONS DR STE 202,HOUSTON,TX,77069,CT,Texas Profit,U,Secretary of State (SOS) File Number,2001-11-20 00:00:00,0,Active or Inactive with no Reason Code,2001-11-20 00:00:00,2001-11-20 00:00:00,A,Active,A,Active,2020-03-25 02:13:35
3,10105495120,"LIL IKES, INC.",7110 AVENUE C,HOUSTON,TX,77011,CT,Texas Profit,U,Secretary of State (SOS) File Number,2001-12-04 00:00:00,0,Active or Inactive with no Reason Code,2001-12-04 00:00:00,2001-12-04 00:00:00,A,Active,A,Active,2020-03-25 02:13:35
4,10105517915,"WOOD GROUP USA, INC.",17325 PARK ROW,HOUSTON,TX,77084,CT,Texas Profit,U,Secretary of State (SOS) File Number,2007-01-01 00:00:00,0,Active or Inactive with no Reason Code,2012-12-22 00:00:00,2012-12-22 00:00:00,A,Active,A,Active,2020-03-25 02:13:35


#### Removing outlier that occured for 2008-1-1

In [15]:
df_franchise_tax_info= df_franchise_tax_info[df_franchise_tax_info['Responsibility_Beginning_Date']!='2008-01-01']
df_franchise_tax_info.count()

Taxpayer_Number                   249217
Taxpayer_Name                     249216
Taxpayer_Address                  249217
Taxpayer_City                     249217
Taxpayer_State                    249217
Taxpayer_Zip_Code                 249217
Taxpayer_Organizational_Type      249217
Taxpayer_Organizational_Name      249217
Record_Type_Code                  249217
Record_Type_Name                  249217
Responsibility_Beginning_Date     249217
Responsibility_End_Reason_Code    249217
Responsibility_End_Reason_Name    249217
SOS_Charter_Date                  241989
SOS_Status_Date                   241989
SOS_Status_Code                   241989
SOS_Status_Name                   241981
Rigth_to_Tansact_Business_Code    249217
Rigth_to_Tansact_Business_Name    249217
DateTime                          249217
dtype: int64

#### Saving to csv file

In [17]:
formattedData = os.path.abspath('HotelOccupancyTaxData/formattedData/HOUFTACT.csv')
df_franchise_tax_info.to_csv(formattedData, index = False)